<a href="https://colab.research.google.com/github/abhijeet06793/MNIST_DL/blob/master/MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **GAN on MNIST Dataset**

GAN is an unsupervised deep learning algorithm where we have a Generator network against an adversarial network called Discriminator network.
Generator generates counterfeit currency. Discriminators are a team of cops trying to detect the counterfeit currency. Counterfeiters and cops both are trying to beat each other at their game.

Both Generator and Discriminator will be multi-layer perceptrons(MLP)

Generator’s objective will be to generate data that is very similar to the training data. Data generated from Generator should be indistinguishable from the real data.
Discriminator takes two sets of input, one input comes from the training dataset(real data) and the other input is the dataset generated by Generator.

**Importing the libraries**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LeakyReLU
from tqdm import tqdm
from tensorflow.keras import Model
from tensorflow.keras import Input

**Importing the MNIST dataset**

In [4]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("Initial shape")
print(x_train.shape)
print(y_train.shape)

x_train = x_train.reshape((60000, 784))/255
x_test = x_test.reshape((10000, 784))/255
print("Final shape")
print(x_train.shape)
print(x_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
Initial shape
(60000, 28, 28)
(60000,)
Final shape
(60000, 784)
(10000, 784)


### Generator

We create Generator which uses MLP using simple dense layers activated by tanh

In [5]:
def create_generator():
  generator = Sequential()
  generator.add(Dense(units=256, input_dim = 100))
  generator.add(LeakyReLU(alpha=0.2))
  generator.add(Dense(units=512))
  generator.add(LeakyReLU(alpha=0.2))
  generator.add(Dense(units=1024))
  generator.add(LeakyReLU(alpha=0.2))

  generator.add(Dense(units=784, activation='tanh'))

  generator.compile(loss = 'binary_crossentropy', optimizer='adam')

  return generator

In [6]:
g = create_generator()
g.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 784)               8

### **Descriminator**

We now create the Discriminator which is also MLP. Discriminator will take the input from real data which is of the size 784 and also the images generated from Generator.


In [7]:
def create_discriminator():
  discriminator = Sequential()
  discriminator.add(Dense(units=1024, input_dim=784))
  discriminator.add(LeakyReLU(alpha=0.2))
  discriminator.add(Dropout(rate = 0.3))

  discriminator.add(Dense(units=512))
  discriminator.add(LeakyReLU(alpha=0.2))
  discriminator.add(Dropout(rate = 0.3))

  discriminator.add(Dense(units=256))
  discriminator.add(LeakyReLU(alpha=0.2))

  discriminator.add(Dense(units=1, activation='sigmoid'))
  discriminator.compile(loss='binary_crossentropy', optimizer='adam')

  return discriminator

d = create_discriminator()
d.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)              

We now create the GAN where we combine the Generator and Discriminator. When we train the generator we will freeze the Discriminator and vice versa.

In [8]:
#Note - We will input the noised image of shape 100 units to the Generator as an Input. 
#The output generated from the Generator(of dim - None,786) will be fed to the Discriminator.

def create_gan(generator, discriminator):
  discriminator.trainable = False #This will froze the discriminator layer
  gan_input = Input(shape=(100,)) #Input is used to initialize a tensor. We have to give a tensor to a model. Also Model class takes a tensor as an input.
  x = generator(gan_input)  #Here x is tensor that is returned of dim (None, 784) and this tensor will act an input to discriminator. 
  gan_output = discriminator(x) #Here gan_output is a tensor.
  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss='binary_crossentropy', optimizer = 'adam')
  return gan

gan = create_gan(g, d)
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 784)               1486352   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


## **Plotting**

Before we start training the model, we will write a function plot_generated_images to plot the generated images. This way we can see how the images are generated. We save the generated images to file that we can view later

In [35]:
def plot_generated_images(epoch, generator):
  noise = np.random.normal(loc=0, scale=1, size=(100,100))
  generated_image = generator.predict(noise)
  generated_image= generated_image.reshape((100, 28, 28))
  fig = plt.figure(figsize=(10,10)) 
  for i in range(0,100):
    plt.subplot(10,10,i+1)
    plt.imshow(generated_image[i], interpolation='nearest', cmap='gray')
    plt.axis('off')

  plt.tight_layout()
  plt.savefig(fname= "/content/Epochs_figure/epoch_{}".format(epoch))
  plt.close(fig)

### **Training**

We finally start to train GAN. We will first have the full code for training GAN and then break it step by step for understanding how the training happens.


In [34]:
def training(epochs=1, batch_size=100):
  
  num_of_batches = int(x_train.shape[0]/batch_size)
  generator = create_generator()
  discriminator = create_discriminator()
  gan_model = create_gan(generator, discriminator)

  for e in range(1, epochs+1):
    print("Epoch-{} :".format(e))
    for j in range(num_of_batches):
      noise_batch = np.random.uniform(0,1,size=(100, 100))
      generated_image = generator.predict(noice_batch)
      image_batch = x_train[j*100:(j+1)*100]

      x = np.concatenate((image_batch, generated_image))
      y = np.zeros(2*batch_size)
      y[0:batch_size] = 1

      





     



  return gan_model


In [46]:
y =np.zeros(2*100)
y[0:100] = 1
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])